# Semantic Search with Amazon OpenSearch 

This is quick demo on how to use Amazon OpeSearch develop semantic search application.

![word vector](word2vec.png)

First, Install required library

In [20]:
!pip install -q transformers
!pip install -q elasticsearch==7.10
!pip install -q boto3
!pip install -q requests
!pip install -q requests-aws4auth
!pip install -q opensearch-py

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


Use Hugging Face BERT model to generate vectorization data, every sentence is 768 dimention data.
![BERT](nlp_bert.png)

In [58]:
import torch
from transformers import AutoTokenizer, AutoModel
from transformers import DistilBertTokenizer, DistilBertModel

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


def sentence_to_vector(raw_inputs):
    #model_name = "distilbert-base-uncased"
    model_name = "sentence-transformers/msmarco-distilbert-base-dot-prod-v3"
    tokenizer = DistilBertTokenizer.from_pretrained(model_name)
    model = DistilBertModel.from_pretrained(model_name)
    inputs_tokens = tokenizer(raw_inputs, padding=True, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model(**inputs_tokens)

    sentence_embeddings = mean_pooling(outputs, inputs_tokens['attention_mask'])
    return sentence_embeddings


Prepare some text content for semantic search

In [59]:
import json
import pandas as pd

def load_pqa(file_name,number_rows=100):
    print("start::")
    qa_list = []
    df = pd.DataFrame(columns=('question', 'answer'))
    with open(file_name) as f:
        i=0
        for line in f:
            data = json.loads(line)
            df.loc[i] = [data['question_text'],data['answers'][0]['answer_text']]
            i+=1
            if(i == number_rows):
                break
    return df


qa_list = load_pqa('amazon_pqa/amazon_pqa_headsets.json',number_rows=1000)


start::


In [60]:
vector_sentences = sentence_to_vector(qa_list["question"].tolist())

Use Python API to set up connection with OpenSearch Cluster

In [61]:
import boto3
from elasticsearch import Elasticsearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
region = 'us-east-2'
service = 'es'
credentials = boto3.Session().get_credentials()
awsauth = ('master', 'password')

es_host = 'search-search-relevance-3jasbt2gyga3rt7rkwawv22yne.us-east-2.es.amazonaws.com'
es_client = Elasticsearch(
   hosts = [{'host': es_host, 'port': 443}],
   http_auth = awsauth,
   use_ssl = True,
   verify_certs = True,
   connection_class = RequestsHttpConnection
)

In [24]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import boto3

host = 'search-search-relevance-3jasbt2gyga3rt7rkwawv22yne.us-east-2.es.amazonaws.com' 
region = 'us-east-2' 

credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region)
index_name = 'nlp_pqa'

es_client = OpenSearch(
    hosts = [{'host': host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

Create a index with 2 fields, the first field is "content" for raw sentece, the second field is "nlp_article_vector" for vector data.

In [62]:
knn_index = {
    "settings": {
        "index.knn": True
    },
    "mappings": {
        "properties": {
            "question_vector": {
                "type": "knn_vector",
                "dimension": 768,
                "store": True
            },
            "question": {
                "type": "text",
                "store": True
            },
            "answer": {
                "type": "text",
                "store": True
            }
        }
    }
}


In [64]:
es_client.indices.delete(index="nlp_pqa")


{'acknowledged': True}

In [65]:
es_client.indices.create(index="nlp_pqa",body=knn_index,ignore=400)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'nlp_pqa'}

Show the created index information

In [66]:
es_client.indices.get(index="nlp_pqa")

{'nlp_pqa': {'aliases': {},
  'mappings': {'properties': {'answer': {'type': 'text', 'store': True},
    'question': {'type': 'text', 'store': True},
    'question_vector': {'type': 'knn_vector',
     'store': True,
     'dimension': 768}}},
  'settings': {'index': {'number_of_shards': '5',
    'provided_name': 'nlp_pqa',
    'knn': 'true',
    'creation_date': '1662559048842',
    'number_of_replicas': '1',
    'uuid': '9nZR2FSWToyFeJMEszQ69A',
    'version': {'created': '7100299'}}}}}

Ingest the raw sentence content and corresponding vector data into OpenSearch index

In [67]:
i = 0
for c in qa_list["question"].tolist():
    content=c
    vector=vector_sentences[i].tolist()
    answer=qa_list["answer"][i]
    i+=1
    es_client.index(index='nlp_pqa',body={"question_vector": vector, "question": content,"answer":answer})

In [68]:
res = es_client.search(index="nlp_pqa", body={"query": {"match_all": {}}})
print("Got %d Hits:" % res['hits']['total']['value'])

Got 1000 Hits:


# This is done for ingesting history data

Generate vector data for the question by calling SageMaker model

In [69]:
query_raw_sentences = ['does this work with xbox?']
query_vector_sentences = sentence_to_vector(query_raw_sentences)
search_vector = query_vector_sentences[0].tolist()


In [70]:
search_vector

[0.5395175218582153,
 -0.1694280207157135,
 0.24309676885604858,
 1.0087178945541382,
 0.2524005174636841,
 -0.3998754918575287,
 0.4078301191329956,
 -0.05830769240856171,
 0.831731379032135,
 -0.39555850625038147,
 0.2095128446817398,
 0.24777638912200928,
 -0.2019232213497162,
 0.08187907189130783,
 0.2887134552001953,
 -0.015251528471708298,
 -0.49697011709213257,
 0.4830368161201477,
 0.09401020407676697,
 -0.24340802431106567,
 0.20390161871910095,
 0.22908106446266174,
 0.5578609704971313,
 0.41232165694236755,
 -0.1333702951669693,
 0.5044822692871094,
 0.5887789130210876,
 -0.5316212773323059,
 -0.021694548428058624,
 -0.1127176582813263,
 0.3756261467933655,
 0.40633976459503174,
 -0.3218729794025421,
 -1.0404669046401978,
 0.12823326885700226,
 -0.10538988560438156,
 -0.14742247760295868,
 0.6923909187316895,
 0.26778164505958557,
 -0.4824138879776001,
 0.06939248740673065,
 -0.09383419156074524,
 0.544048547744751,
 0.1967717558145523,
 0.42453762888908386,
 -0.471453517675

Show the vector data of the question

Search the data with one query:

In [78]:
query={
    "size": 50,
    "query": {
        "knn": {
            "question_vector":{
                "vector":search_vector,
                "k":50
            }
        }
    }
}

res = es_client.search(index="nlp_pqa", 
                       body=query,
                       stored_fields=["question","answer"])
#print("Got %d Hits:" % res['hits']['total']['value'])
query_result=[]
for hit in res['hits']['hits']:
    #print(hit)
    #print("score:",hit['_score'])
    #print("question:", hit['fields']['question'])
    #print("answer:",hit['fields']['answer'])
    #print("\n----------------------------------\n")
    row=[hit['_id'],hit['_score'],hit['fields']['question'][0],hit['fields']['answer'][0]]
    query_result.append(row)

query_result_df = pd.DataFrame(data=query_result,columns=["_id","_score","question","answer"])
display(query_result_df)

,_id,_score,question,answer
0,czE-GIMB4nMnV5QpD1MO,0.036350,Does this work with the xbox one?,"Yeah of course , but you must have an adapter ..."
1,ZzE9GIMB4nMnV5Qp81HZ,0.035702,Does this work with xbox one?,"sorry, Im not an xbox user."
2,5TE-GIMB4nMnV5QpB1LG,0.026802,does this work on xbox one?,"I'm sorry, but not!"
3,xDE-GIMB4nMnV5QpBlIT,0.025834,will these work with xbox one?,Yes
4,3TE9GIMB4nMnV5Qp7FDh,0.022872,Does this work for xbox one S?,It should work.
5,9TE-GIMB4nMnV5QpCFKQ,0.022815,Does it work for xbox one?,"Thanks for your inquiry, it just works with PS..."
6,vDE-GIMB4nMnV5QpBVKi,0.022377,will this work with xbox one,"Diane, you would also need to purchase an Xbox..."
7,VjE9GIMB4nMnV5Qp8lH8,0.022370,Will this work for Xbox 360?,"NO, but it works for Xbox one slim/X and other..."
8,6TE9GIMB4nMnV5Qp7VB9,0.022133,Do they work with xbox one?,"No they don't , but let's hope that Microsoft ..."
9,7jE-GIMB4nMnV5QpCFI3,0.021073,Does it work for Xbox 360?,"Sorry , it can't .Just for PS4"


Search the data with another query:

In [77]:
query={
    "size": 50,
    "query": {
        "match": {
            "question":"does this work with xbox?"
        }
    }
}

res = es_client.search(index="nlp_pqa", 
                       body=query,
                       stored_fields=["question","answer"])
#print("Got %d Hits:" % res['hits']['total']['value'])
query_result=[]
for hit in res['hits']['hits']:
    row=[hit['_id'],hit['_score'],hit['fields']['question'][0],hit['fields']['answer'][0]]
    query_result.append(row)

query_result_df = pd.DataFrame(data=query_result,columns=["_id","_score","question","answer"])
display(query_result_df)


,_id,_score,question,answer
0,ZzE9GIMB4nMnV5Qp81HZ,9.973280,Does this work with xbox one?,"sorry, Im not an xbox user."
1,5TE-GIMB4nMnV5QpB1LG,8.363104,does this work on xbox one?,"I'm sorry, but not!"
2,czE-GIMB4nMnV5QpD1MO,8.347527,Does this work with the xbox one?,"Yeah of course , but you must have an adapter ..."
3,3TE9GIMB4nMnV5Qp7FDh,7.698657,Does this work for xbox one S?,It should work.
4,aDE-GIMB4nMnV5QpHFRd,7.208141,Can this work with an xbox one with the adpate...,Yes but the audio is not very good. I recommen...
5,vDE-GIMB4nMnV5QpBVKi,7.138710,will this work with xbox one,"Diane, you would also need to purchase an Xbox..."
6,7jE-GIMB4nMnV5QpCFI3,7.115393,Does it work for Xbox 360?,"Sorry , it can't .Just for PS4"
7,9TE-GIMB4nMnV5QpCFKQ,7.115393,Does it work for xbox one?,"Thanks for your inquiry, it just works with PS..."
8,vTE-GIMB4nMnV5QpBVKx,7.031937,Does this work with PS4,yes
9,CzE-GIMB4nMnV5QpF1Rw,7.031937,Does this work with ps4,Yes it does


You can also use OpenSearch Dashboard Dev Tools to run the following query:

<code>
GET nlp_pqa/_search
{
  "size": 10,
  "query": {
      "knn": {
          "question_vector":{
              "vector":[0.11331528425216675,
 -0.149199441075325,
 -0.042027559131383896,
 0.2201102077960968,
 0.29214081168174744,
 -0.4763796031475067,
 -0.05931148678064346,
 0.2968612015247345,
 0.18734148144721985,
 -0.09313250333070755,
 0.28115445375442505,
 -0.2739621698856354,
 -0.3604494333267212,
 0.1875743567943573,
 -0.1513073593378067,
 -0.0512186698615551,
 0.08968539535999298,
 0.3044019639492035,
 -0.14454962313175201,
 0.17908819019794464,
 -0.06742167472839355,
 0.11772555112838745,
 -0.08429372310638428,
 0.308734267950058,
 -0.1802659034729004,
 0.15799733996391296,
 0.274673193693161,
 0.03770603984594345,
 -0.2029426246881485,
 -0.006454922258853912,
 0.2704549729824066,
 0.12450575083494186,
 -0.19017867743968964,
 -0.3293623924255371,
 -0.20594549179077148,
 -0.19873106479644775,
 -0.23891012370586395,
 0.15369676053524017,
 0.07393668591976166,
 0.08634427189826965,
 -0.4533677399158478,
 -0.21077948808670044,
 0.1520564705133438,
 0.19249163568019867,
 0.22398707270622253,
 -0.20696260035037994,
 0.12574002146720886,
 -0.07689037919044495,
 -0.06795704364776611,
 -0.06153655797243118,
 -0.363738089799881,
 0.12755617499351501,
 0.04556633532047272,
 0.053204260766506195,
 0.2677585184574127,
 0.15453489124774933,
 0.017572317272424698,
 -0.38331857323646545,
 0.049343325197696686,
 0.08259770274162292,
 0.12977486848831177,
 -0.11843208223581314,
 -0.32934680581092834,
 -0.3848985731601715,
 0.45919737219810486,
 -0.04859811067581177,
 -0.14126302301883698,
 0.6306865215301514,
 -0.48285403847694397,
 -0.2022354155778885,
 -0.024578865617513657,
 -0.42419925332069397,
 -0.14303231239318848,
 -0.12628792226314545,
 -0.09115435183048248,
 0.37926051020622253,
 -0.17953065037727356,
 0.5672749876976013,
 0.5427437424659729,
 -0.28916406631469727,
 -0.1706433892250061,
 0.28411126136779785,
 0.2287624478340149,
 0.4085281193256378,
 0.2006869614124298,
 0.05593251809477806,
 -0.29632043838500977,
 0.1123407781124115,
 -0.08746325969696045,
 0.5961596369743347,
 0.09390804171562195,
 -0.2431793212890625,
 0.25015556812286377,
 0.2078317552804947,
 0.37671127915382385,
 -0.05198490992188454,
 -0.15028096735477448,
 0.14881953597068787,
 0.12312406301498413,
 0.4514046609401703,
 0.10177795588970184,
 -0.5412963628768921,
 -0.4133792519569397,
 0.3056791424751282,
 -0.08113449066877365,
 0.08455594629049301,
 0.18314138054847717,
 0.22526076436042786,
 -0.16980767250061035,
 0.08473488688468933,
 0.17627808451652527,
 0.11239004135131836,
 0.26718416810035706,
 0.14943872392177582,
 -0.4256228804588318,
 0.2182038277387619,
 -0.02536270022392273,
 0.004092143848538399,
 0.035331856459379196,
 -0.10238410532474518,
 0.19215039908885956,
 0.17645832896232605,
 0.13557423651218414,
 0.9306938648223877,
 -0.1937524527311325,
 0.26337698101997375,
 -0.23760700225830078,
 0.12201821804046631,
 0.35801804065704346,
 -0.4088429808616638,
 0.4996858537197113,
 0.663728654384613,
 0.5409157276153564,
 0.10714609920978546,
 -0.06056676059961319,
 0.30662766098976135,
 -0.02238406613469124,
 -0.031213589012622833,
 -0.7654272317886353,
 0.08468995988368988,
 -0.04519497603178024,
 0.3623118996620178,
 -0.10696031153202057,
 -0.25337791442871094,
 0.28772056102752686,
 0.3174844980239868,
 -0.059575073421001434,
 -0.4179319143295288,
 -0.11264122277498245,
 -0.07549981772899628,
 -0.09075874090194702,
 0.4101727604866028,
 -0.32003310322761536,
 -0.41433635354042053,
 -0.12219682335853577,
 0.2795346975326538,
 -0.0944761261343956,
 -0.12720000743865967,
 0.16714942455291748,
 0.08589955419301987,
 0.2958478033542633,
 -0.07993771880865097,
 -0.10300196707248688,
 -0.03264104574918747,
 -0.172191321849823,
 0.19987039268016815,
 0.35919633507728577,
 0.29606491327285767,
 0.10812994837760925,
 0.3187010586261749,
 -0.036915477365255356,
 0.20325395464897156,
 0.5364779233932495,
 0.11619143187999725,
 0.026240818202495575,
 0.10451950877904892,
 0.17127425968647003,
 0.2791525721549988,
 0.21741853654384613,
 0.3094721734523773,
 -1.3558069467544556,
 -0.20675751566886902,
 0.13220597803592682,
 -0.16281072795391083,
 0.11867393553256989,
 -0.021726027131080627,
 0.2130310833454132,
 -0.2894837558269501,
 -0.19038474559783936,
 0.09313315153121948,
 -0.42601659893989563,
 -0.13613560795783997,
 -0.3351694941520691,
 0.05970066413283348,
 0.49833059310913086,
 -0.5758011341094971,
 -0.29661035537719727,
 -0.2528771162033081,
 0.27864202857017517,
 -0.02609999291598797,
 0.025837982073426247,
 -0.18093204498291016,
 0.07253424823284149,
 -0.06696362793445587,
 -0.5573418140411377,
 0.04354557394981384,
 -0.19504867494106293,
 -0.21730002760887146,
 -0.6129741668701172,
 0.1297992467880249,
 -0.4562245011329651,
 0.19173038005828857,
 -0.026043951511383057,
 -0.1481418013572693,
 0.2824363708496094,
 0.03253085911273956,
 -0.14274179935455322,
 -0.05328623205423355,
 -0.23304760456085205,
 0.14029699563980103,
 0.010882183909416199,
 -0.1366325169801712,
 -0.3010956943035126,
 0.4080979526042938,
 -0.14301082491874695,
 0.5213117003440857,
 0.15125709772109985,
 0.054458193480968475,
 0.2074461281299591,
 -0.20393310487270355,
 -0.3852587640285492,
 0.012760629877448082,
 0.02575640380382538,
 0.13618095219135284,
 -0.15962626039981842,
 0.04871378466486931,
 -0.2401711493730545,
 -0.202806293964386,
 -0.09212779253721237,
 -0.07248768955469131,
 -0.3092164993286133,
 0.6895079016685486,
 0.38291290402412415,
 0.15165352821350098,
 0.22793427109718323,
 -0.00892980769276619,
 0.010221598669886589,
 0.06654109060764313,
 -0.06181803345680237,
 -0.17048993706703186,
 -0.19254541397094727,
 -0.380470335483551,
 -0.151157945394516,
 -0.30914413928985596,
 0.06046297401189804,
 0.035200152546167374,
 -0.12414196133613586,
 0.1531936377286911,
 -0.31038784980773926,
 0.18896391987800598,
 0.4319060742855072,
 0.25042951107025146,
 0.11430439352989197,
 0.09273344278335571,
 -0.2374401092529297,
 -0.7735293507575989,
 0.06447669863700867,
 0.4399068355560303,
 0.29475200176239014,
 -0.0670214369893074,
 -0.11792901903390884,
 -0.49522772431373596,
 0.17174963653087616,
 -0.2877271771430969,
 -0.38811665773391724,
 -0.2653396725654602,
 0.32558661699295044,
 0.08573565632104874,
 -0.0769907683134079,
 -0.12483366578817368,
 -0.08663171529769897,
 0.8187874555587769,
 -0.490420937538147,
 0.253390371799469,
 0.07564549893140793,
 -0.11540060490369797,
 -0.02992701530456543,
 -0.5377054214477539,
 -0.16783778369426727,
 -0.5102713704109192,
 0.017989827319979668,
 -0.07273252308368683,
 -0.22196392714977264,
 -0.08788725733757019,
 0.4497719705104828,
 -0.22582615911960602,
 0.3652379512786865,
 0.12551188468933105,
 -0.16521084308624268,
 -0.24213570356369019,
 0.048018526285886765,
 -0.35700589418411255,
 0.21551688015460968,
 0.14000722765922546,
 -0.18247771263122559,
 0.08657519519329071,
 -0.14076586067676544,
 -0.24103601276874542,
 -3.2576448917388916,
 0.006805450655519962,
 -0.10996705293655396,
 -0.06416475772857666,
 0.16446813941001892,
 -0.1809379756450653,
 -0.07074707746505737,
 0.06054804474115372,
 -0.5302340388298035,
 -0.015478145331144333,
 -0.051932353526353836,
 -0.12556931376457214,
 0.09983156621456146,
 0.24560321867465973,
 0.09695985913276672,
 0.003226742148399353,
 0.3159646987915039,
 -0.19122432172298431,
 -0.40202566981315613,
 -0.11386746168136597,
 -0.42993760108947754,
 -0.051110535860061646,
 0.144333615899086,
 -0.2121087908744812,
 0.17408210039138794,
 0.5687850117683411,
 -0.09723345935344696,
 -0.02542771026492119,
 0.12265361845493317,
 0.010100634768605232,
 -0.3021245300769806,
 -0.023458313196897507,
 0.10586538910865784,
 0.4714823067188263,
 -0.07992628216743469,
 -0.2868834137916565,
 0.015308305621147156,
 -0.043809644877910614,
 0.1647900640964508,
 -0.2166755497455597,
 -0.17523600161075592,
 0.22506725788116455,
 0.1997949331998825,
 0.05231878533959389,
 0.435850590467453,
 -0.5309421420097351,
 0.16183730959892273,
 0.21877899765968323,
 -0.060341015458106995,
 0.19556961953639984,
 0.0019241124391555786,
 -0.15309663116931915,
 -0.07082498073577881,
 -0.12079646438360214,
 -0.4912208020687103,
 -0.5602697730064392,
 0.16692331433296204,
 0.432321161031723,
 0.0196707621216774,
 -0.36544567346572876,
 0.10117261111736298,
 -0.10999829322099686,
 -0.26756495237350464,
 0.17708925902843475,
 -0.08364424109458923,
 -0.560991108417511,
 -0.5037732124328613,
 -0.4856431782245636,
 0.20751671493053436,
 0.21836687624454498,
 -0.11867127567529678,
 0.24950888752937317,
 -0.17112916707992554,
 -1.0588915348052979,
 -0.2074393481016159,
 0.0581999234855175,
 -0.05870150402188301,
 -0.22864805161952972,
 0.031854186207056046,
 0.013091369532048702,
 -0.1532750427722931,
 -0.17551100254058838,
 -0.1273740977048874,
 0.004304900765419006,
 -0.4737584590911865,
 -0.3407425582408905,
 0.0686853751540184,
 -0.14436833560466766,
 -0.08427019417285919,
 0.200969398021698,
 -0.09410513192415237,
 0.1459212601184845,
 0.3633514642715454,
 0.38368579745292664,
 -0.0036943890154361725,
 0.21510049700737,
 0.35987499356269836,
 -0.30357974767684937,
 0.15750433504581451,
 -0.12864266335964203,
 0.24018999934196472,
 -0.25836634635925293,
 0.2777301073074341,
 -0.32564330101013184,
 0.017872733995318413,
 0.03780350089073181,
 -0.252519816160202,
 0.19924400746822357,
 0.1125449538230896,
 0.010189900174736977,
 -0.040505532175302505,
 -0.5340555906295776,
 0.5962439179420471,
 -0.35759809613227844,
 0.02189571224153042,
 -0.14631378650665283,
 0.4151935279369354,
 0.442879319190979,
 0.10922059416770935,
 0.0579012855887413,
 -0.29864078760147095,
 0.8076953887939453,
 -0.09876681864261627,
 -0.42895033955574036,
 -0.3667493164539337,
 0.0689033567905426,
 -0.09455326944589615,
 -0.06755158305168152,
 0.19522559642791748,
 -0.14579664170742035,
 -0.040403131395578384,
 -0.07084741443395615,
 -0.0541570819914341,
 0.1973528265953064,
 -0.006280157715082169,
 -0.3793770670890808,
 -0.24379152059555054,
 0.3361445367336273,
 0.06586570292711258,
 0.1112912967801094,
 -0.2088366001844406,
 0.07139995694160461,
 -0.08862830698490143,
 0.02404431253671646,
 -0.07779435813426971,
 0.18833735585212708,
 0.16973789036273956,
 -0.12524433434009552,
 -0.09160379320383072,
 0.18475845456123352,
 -0.31650814414024353,
 -0.385364830493927,
 -0.24206367135047913,
 -0.1388472616672516,
 0.24685761332511902,
 -0.05366567522287369,
 0.3989082872867584,
 -0.06169511750340462,
 0.24866585433483124,
 -0.4922820031642914,
 0.05517095699906349,
 0.039194781333208084,
 -0.21572421491146088,
 0.43271154165267944,
 0.09567864239215851,
 0.05856385827064514,
 -0.06618662923574448,
 -0.30561986565589905,
 -0.3714120388031006,
 0.10801568627357483,
 -0.16792170703411102,
 -0.30374613404273987,
 0.10797885060310364,
 -0.17577101290225983,
 0.07830900698900223,
 0.3014753460884094,
 -0.017140667885541916,
 -0.35539954900741577,
 -0.19169887900352478,
 0.016435861587524414,
 0.17972147464752197,
 -0.21973592042922974,
 -0.2047157883644104,
 -0.3010421395301819,
 0.3283456563949585,
 -0.16672612726688385,
 -0.012128744274377823,
 -0.042837757617235184,
 0.13738611340522766,
 -0.019291453063488007,
 -0.17817199230194092,
 0.2522180676460266,
 -0.265263170003891,
 -0.25404608249664307,
 -0.5514754056930542,
 -0.15070275962352753,
 0.32102420926094055,
 0.016712944954633713,
 0.20063534379005432,
 -0.07386936247348785,
 0.172491654753685,
 0.15463320910930634,
 -0.21262764930725098,
 0.24408112466335297,
 0.27759742736816406,
 0.03816591575741768,
 0.08766619861125946,
 0.30831673741340637,
 -0.13697925209999084,
 -0.1404787302017212,
 -0.21030324697494507,
 -0.2844001352787018,
 -0.3427157998085022,
 -0.05639936402440071,
 0.11203678697347641,
 0.18808045983314514,
 0.22180891036987305,
 -0.1642322540283203,
 -0.024602442979812622,
 0.01945117861032486,
 -0.7126460075378418,
 -0.1166064441204071,
 0.23284141719341278,
 0.11934860050678253,
 -0.4260438084602356,
 -0.5733505487442017,
 0.12037938088178635,
 0.13922922313213348,
 -0.27545225620269775,
 -0.12336248904466629,
 0.33474618196487427,
 -0.24044039845466614,
 0.07871866971254349,
 0.3009663224220276,
 -0.02062249928712845,
 0.2188863307237625,
 -0.47363534569740295,
 -0.05345749482512474,
 0.03316528722643852,
 -0.4257044792175293,
 -0.3080853521823883,
 -0.1585712879896164,
 -0.07998616993427277,
 0.05640851706266403,
 0.14354562759399414,
 -0.1443110704421997,
 -0.3613317906856537,
 0.24981847405433655,
 -0.3713478147983551,
 -0.07477186620235443,
 -0.23013144731521606,
 0.024237345904111862,
 0.004744056612253189,
 -0.04180505499243736,
 -0.08386224508285522,
 -0.8431572318077087,
 -0.2767137587070465,
 -0.1611274927854538,
 -0.43179309368133545,
 -0.2249959409236908,
 -0.19520661234855652,
 0.08197978138923645,
 0.2737003564834595,
 0.4135475158691406,
 0.10402265936136246,
 0.13194844126701355,
 0.38253048062324524,
 -0.08772691339254379,
 0.1416245996952057,
 -0.059250034391880035,
 -0.3436194062232971,
 0.060655999928712845,
 0.06816509366035461,
 0.12182965874671936,
 -0.1739179790019989,
 -0.15613749623298645,
 0.269167959690094,
 -0.6043237447738647,
 0.004751473665237427,
 -0.13860370218753815,
 0.19166260957717896,
 0.26361799240112305,
 0.005099224392324686,
 0.06560513377189636,
 0.2496124655008316,
 -0.3822724521160126,
 0.18326234817504883,
 0.3295528292655945,
 0.12993061542510986,
 -0.07174065709114075,
 -0.07332970201969147,
 0.3340991735458374,
 0.22943620383739471,
 -0.17639334499835968,
 0.19916416704654694,
 0.08508304506540298,
 0.006985129788517952,
 -0.2353828400373459,
 -0.032489895820617676,
 -0.09909343719482422,
 -0.2058390974998474,
 0.4797382056713104,
 0.45118004083633423,
 0.49668174982070923,
 -0.11154557764530182,
 0.19885167479515076,
 -0.12510204315185547,
 0.021588707342743874,
 -0.27016764879226685,
 0.04478210583329201,
 -0.2959427535533905,
 -0.07937052100896835,
 0.32096171379089355,
 0.5367889404296875,
 -0.3119184374809265,
 0.02850569598376751,
 0.45323318243026733,
 -0.19138577580451965,
 -0.1020713746547699,
 0.1782682240009308,
 0.01456280704587698,
 -0.17806318402290344,
 0.41746553778648376,
 0.16218940913677216,
 0.12526237964630127,
 0.16393232345581055,
 -0.27241718769073486,
 -0.32347238063812256,
 0.1677711457014084,
 0.3261127769947052,
 0.17178666591644287,
 0.0731237456202507,
 -0.11623422801494598,
 0.4175044894218445,
 -0.12431147694587708,
 -0.2621106803417206,
 -0.011462725698947906,
 0.07356804609298706,
 -0.004403345286846161,
 0.11787042021751404,
 -0.07267637550830841,
 0.2030121237039566,
 0.45271793007850647,
 0.15177910029888153,
 0.05076213926076889,
 0.15170007944107056,
 0.11181080341339111,
 -0.1495269536972046,
 0.34939318895339966,
 0.5645960569381714,
 0.09819833934307098,
 0.06843650341033936,
 -0.19271016120910645,
 0.3615749776363373,
 0.11058712750673294,
 -0.16253066062927246,
 0.8316932916641235,
 0.4661566913127899,
 0.3206743896007538,
 -0.18964922428131104,
 0.03493208438158035,
 -0.2133077085018158,
 0.7254759073257446,
 -0.5912052392959595,
 0.07881878316402435,
 0.292021781206131,
 0.27907153964042664,
 0.14043329656124115,
 -0.20757044851779938,
 -0.20912687480449677,
 -0.04417885094881058,
 0.3457914888858795,
 0.3140830099582672,
 0.0015315264463424683,
 0.21846160292625427,
 0.6040800213813782,
 0.13886728882789612,
 -0.11379222571849823,
 -0.10611830651760101,
 -0.23417459428310394,
 0.09799261391162872,
 0.1858253926038742,
 -0.5992369651794434,
 -0.1822013258934021,
 -0.21808083355426788,
 -0.4037099778652191,
 0.07929840683937073,
 -0.12268958985805511,
 0.3326212465763092,
 -0.5145058631896973,
 0.14936985075473785,
 -0.29830339550971985,
 0.16000233590602875,
 -0.4564428925514221,
 0.05875798314809799,
 -0.38142430782318115,
 -0.06025560945272446,
 0.09610933065414429,
 0.40518897771835327,
 0.053778983652591705,
 0.43829667568206787,
 -0.38093796372413635,
 0.019991211593151093,
 0.06325417011976242,
 -0.08941631019115448,
 0.30389168858528137,
 0.19888904690742493,
 0.09366429597139359,
 -0.6110355854034424,
 -0.13659600913524628,
 0.232766792178154,
 0.19396783411502838,
 -0.7474576830863953,
 -0.4083305895328522,
 -0.21520204842090607,
 0.126747265458107,
 0.509273886680603,
 0.04486703500151634,
 -0.1228824034333229,
 0.28429704904556274,
 -0.4000059962272644,
 -0.13006651401519775,
 0.4122338891029358,
 -0.033341191709041595,
 0.040653426200151443,
 0.3157860040664673,
 0.16979454457759857,
 -0.22078324854373932,
 -0.20725591480731964,
 -0.049982212483882904,
 -0.23317529261112213,
 0.31693384051322937,
 0.19512584805488586,
 0.18663930892944336,
 0.2190432846546173,
 -0.287519633769989,
 -0.19005395472049713,
 0.07502634078264236,
 0.053403157740831375,
 -0.5322630405426025,
 -0.10197664797306061,
 0.2749122679233551,
 -0.10590927302837372,
 0.09225264936685562,
 -0.676098108291626,
 0.13152848184108734,
 -0.4378671646118164,
 -0.2646966874599457,
 -0.029934030026197433,
 -0.29750221967697144,
 0.13591934740543365,
 0.12362122535705566,
 0.1184035986661911,
 -0.4014938473701477,
 0.2162323296070099,
 0.11062789708375931,
 0.14721789956092834,
 -0.5236157774925232,
 0.0473502017557621,
 0.1603550761938095],
              "k":10
            }
        }
  },
  "fields": [
    "question",
    "answer"
    ],
  "_source": false
}
</code>